In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [2]:
import torch

print(torch.cuda.device_count())  # should print 2
print(torch.cuda.get_device_name(0))

2
NVIDIA RTX A6000


In [3]:
!nvidia-smi

Thu Nov 13 17:06:15 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A6000    On   | 00000000:1A:00.0 Off |                  Off |
| 30%   31C    P8    21W / 300W |      8MiB / 48685MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A6000    On   | 00000000:1B:00.0 Off |                  Off |
| 30%   

In [4]:
import ultralytics

print("Ultralytics version:", ultralytics.__version__)
print("Using ultralytics from:", ultralytics.__file__)

Ultralytics version: 8.3.195
Using ultralytics from: /home/250027044/ultralyticsWithEfficientNetB3/ultralytics/__init__.py


In [5]:
from ultralytics.nn import tasks

print("SPDADown in globals?", "SPDADown" in tasks.__dict__)
print("MyHGBlock in globals?", "MyHGBlock" in tasks.__dict__)

SPDADown in globals? True
MyHGBlock in globals? True


In [6]:
from ultralytics.nn.modules import Adapter, FABlock, SPDADown

print("Custom modules loaded:", SPDADown, FABlock, Adapter)

Custom modules loaded: <class 'ultralytics.nn.modules.hybrid_blocks.SPDADown'> <class 'ultralytics.nn.modules.hybrid_blocks.FABlock'> <class 'ultralytics.nn.modules.hybrid_blocks.Adapter'>


In [7]:
import os

cfg = "ultralytics/cfg/models/custom/custom_yolov9.yaml"
print("Config exists:", os.path.exists(cfg))

Config exists: True


In [8]:
import torch

from ultralytics.nn.modules.custom_layers import ASPP, CBAM, BiFPN, DropBlock

x = torch.randn(1, 256, 64, 64)
print(CBAM(256)(x).shape)
print(ASPP(256)(x).shape)
print(DropBlock(0.1)(x).shape)

# BiFPN expects a list of 3 feature maps:
bifpn = BiFPN(256)
features = [torch.randn(1, 256, s, s) for s in [80, 40, 20]]
print([f.shape for f in bifpn(features)])

torch.Size([1, 256, 64, 64])
torch.Size([1, 256, 64, 64])
torch.Size([1, 256, 64, 64])
[torch.Size([1, 256, 80, 80]), torch.Size([1, 256, 40, 40]), torch.Size([1, 256, 20, 20])]


In [9]:
# ablation Study

import torch

from ultralytics.nn.tasks import DetectionModel

# cfg = "ultralytics/cfg/models/ablation/00_baseline.yaml"
# cfg = "ultralytics/cfg/models/ablation/01_baseline_plus_efficientnet.yaml"
# cfg = "ultralytics/cfg/models/ablation/02_baseline_plus_fablocks.yaml"
cfg = "ultralytics/cfg/models/ablation/03_baseline_plus_hgblocks.yaml"
# cfg = "ultralytics/cfg/models/ablation/04_baseline_plus_spdadown.yaml"
# cfg = "ultralytics/cfg/models/ablation/05_baseline_plus_dropblock.yaml"
# cfg = "ultralytics/cfg/models/ablation/06_baseline_plus_cbam.yaml"
# cfg = "ultralytics/cfg/models/ablation/07_baseline_plus_bifpn.yaml"
# cfg = "ultralytics/cfg/models/ablation/08_baseline_plus_context.yaml"
# cfg = "ultralytics/cfg/models/ablation/09_full_model.yaml"
model = DetectionModel(cfg=cfg, nc=30)

x = torch.randn(1, 3, 640, 640)
y = model(x)
print("✅ Model runs successfully! Output shapes:")
for out in y if isinstance(y, list) else [y]:
    print(out.shape)

Overriding model.yaml nc=80 with nc=30

                   from  n    params  module                                       arguments                     
  0                  -1  1     37120  ultralytics.nn.modules.conv.Conv             [1280, 3, 2]                  
  1                  -1  1         0  ultralytics.nn.modules.hybrid_blocks.Adapter [1280, 256]                   
  2                  -1  1         0  torch.nn.modules.linear.Identity             []                            
  3                  -1  1         0  ultralytics.nn.modules.hybrid_blocks.MyHGBlock[256, 256, 3, 1]              
  4                  -1  1    885504  ultralytics.nn.modules.conv.Conv             [384, 3, 2]                   
  5                  -1  1         0  torch.nn.modules.linear.Identity             []                            
  6                  -1  1         0  ultralytics.nn.modules.hybrid_blocks.MyHGBlock[384, 384, 3, 1]              
  7                  -1  1         0  ultralyt

In [10]:
!pip install pandas

In [2]:
import os
from datetime import datetime

import pandas as pd
import yaml

from ultralytics import YOLO

# List all your ablation configs
configs = [
    "ultralytics/cfg/models/ablation/00_baseline.yaml",
    "ultralytics/cfg/models/ablation/01_baseline_plus_efficientnet.yaml",
    "ultralytics/cfg/models/ablation/02_baseline_plus_fablocks.yaml",
    "ultralytics/cfg/models/ablation/03_baseline_plus_hgblocks.yaml",
    "ultralytics/cfg/models/ablation/04_baseline_plus_spdadown.yaml",
    "ultralytics/cfg/models/ablation/05_baseline_plus_dropblock.yaml",
    "ultralytics/cfg/models/ablation/06_baseline_plus_cbam.yaml",
    "ultralytics/cfg/models/ablation/07_baseline_plus_bifpn.yaml",
    "ultralytics/cfg/models/ablation/08_baseline_plus_context.yaml",
    "ultralytics/cfg/models/ablation/09_full_model.yaml",
]

results = []
DATASET = "charging-ports-7/data.yaml"
EPOCHS = 25
BATCH = 16
IMGSZ = 640
DEVICE = "0,1"  # your booked GPUs

for cfg in configs:
    model_name = os.path.splitext(os.path.basename(cfg))[0]
    print(f"\n🚀 Training variant: {model_name}\n{'=' * 60}")

    model = YOLO(cfg)
    r = model.train(
        data=DATASET,
        epochs=EPOCHS,
        batch=BATCH,
        imgsz=IMGSZ,
        device=DEVICE,
        project="runs/ablation",
        name=model_name,
        exist_ok=True,
    )

    # Collect results from the last training
    results_file = f"runs/ablation/{model_name}/results.yaml"
    if os.path.exists(results_file):
        with open(results_file) as f:
            d = yaml.safe_load(f)
            results.append(
                {
                    "model": model_name,
                    "mAP50": d.get("metrics/mAP50(B)", None),
                    "mAP50-95": d.get("metrics/mAP50-95(B)", None),
                    "params": d.get("model/parameters", None),
                    "flops": d.get("model/FLOPs", None),
                    "train_time": d.get("train/epochs", None),
                    "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                }
            )

# Save consolidated ablation results
df = pd.DataFrame(results)
df.to_csv("runs/ablation/ablation_results.csv", index=False)
print("\n✅ Ablation study complete! Results saved to runs/ablation/ablation_results.csv")
print(df)


🚀 Training variant: 00_baseline
New https://pypi.org/project/ultralytics/8.3.227 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.195 🚀 Python-3.10.18 torch-2.7.1+cu118 CUDA:0 (NVIDIA RTX A6000, 48685MiB)
                                                        CUDA:1 (NVIDIA RTX A6000, 48685MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=charging-ports-7/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode

W1110 08:19:35.413000 2220238 /home/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/site-packages/torch/distributed/elastic/agent/server/api.py:719] Received Signals.SIGINT death signal, shutting down workers
W1110 08:19:35.415000 2220238 /home/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/site-packages/torch/distributed/elastic/multiprocessing/api.py:900] Sending process 2220241 closing signal SIGINT
W1110 08:19:35.415000 2220238 /home/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/site-packages/torch/distributed/elastic/multiprocessing/api.py:900] Sending process 2220242 closing signal SIGINT
[rank1]: Traceback (most recent call last):
[rank1]:   File "/users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/site-packages/torch/distributed/c10d_logger.py", line 81, in wrapper
[rank1]:     return func(*args, **kwargs)
[rank1]:   File "/users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/si

KeyboardInterrupt: 

[rank0]: Traceback (most recent call last):
[rank0]:   File "/users/2/250027044/.config/Ultralytics/DDP/_temp_xhtlraa1139857772249584.py", line 13, in <module>
[rank0]:   File "/home/250027044/ultralyticsWithEfficientNetB3/ultralytics/engine/trainer.py", line 230, in train
[rank0]:     self._do_train(world_size)
[rank0]:   File "/home/250027044/ultralyticsWithEfficientNetB3/ultralytics/engine/trainer.py", line 349, in _do_train
[rank0]:     self._setup_train(world_size)
[rank0]:   File "/home/250027044/ultralyticsWithEfficientNetB3/ultralytics/engine/trainer.py", line 286, in _setup_train
[rank0]:     self.amp = torch.tensor(check_amp(self.model), device=self.device)
[rank0]:   File "/home/250027044/ultralyticsWithEfficientNetB3/ultralytics/utils/checks.py", line 792, in check_amp
[rank0]:     assert amp_allclose(YOLO("yolo11n.pt"), im)
[rank0]:   File "/home/250027044/ultralyticsWithEfficientNetB3/ultralytics/utils/checks.py", line 780, in amp_allclose
[rank0]:     a = m(batch, imgsz=

In [13]:
import glob

files = glob.glob("runs/ablation/**/*", recursive=True)
for f in files:
    if "result" in f.lower():
        print(f)

runs/ablation/ablation_results.csv
runs/ablation/06_baseline_plus_cbam/results.csv
runs/ablation/06_baseline_plus_cbam/results.png
runs/ablation/04_baseline_plus_spdadown/results.csv
runs/ablation/04_baseline_plus_spdadown/results.png
runs/ablation/09_full_model/results.csv
runs/ablation/09_full_model/results.png
runs/ablation/02_baseline_plus_fablocks/results.csv
runs/ablation/02_baseline_plus_fablocks/results.png
runs/ablation/07_baseline_plus_bifpn/results.csv
runs/ablation/07_baseline_plus_bifpn/results.png
runs/ablation/05_baseline_plus_dropblock/results.csv
runs/ablation/05_baseline_plus_dropblock/results.png
runs/ablation/00_baseline/results.csv
runs/ablation/00_baseline/results.png
runs/ablation/08_baseline_plus_context/results.csv
runs/ablation/08_baseline_plus_context/results.png
runs/ablation/03_baseline_plus_hgblocks/results.csv
runs/ablation/03_baseline_plus_hgblocks/results.png
runs/ablation/01_baseline_plus_efficientnet/results.csv
runs/ablation/01_baseline_plus_efficien

In [14]:
import json
import os
from datetime import datetime

import pandas as pd
import yaml

results = []

for cfg in configs:
    model_name = os.path.splitext(os.path.basename(cfg))[0]
    base_path = f"runs/ablation/{model_name}"

    # Try all possible result files
    possible_files = [
        os.path.join(base_path, "results.yaml"),
        os.path.join(base_path, "results.csv"),
        os.path.join(base_path, "results.json"),
    ]

    found = False
    metrics = {}

    for file in possible_files:
        if os.path.exists(file):
            found = True
            print(f"✅ Found results for {model_name}: {file}")

            if file.endswith(".yaml"):
                with open(file) as f:
                    metrics = yaml.safe_load(f)
            elif file.endswith(".json"):
                with open(file) as f:
                    metrics = json.load(f)
            elif file.endswith(".csv"):
                df_metrics = pd.read_csv(file)
                metrics = df_metrics.iloc[-1].to_dict()  # Last epoch

            results.append(
                {
                    "model": model_name,
                    "mAP50": metrics.get("metrics/mAP50(B)", metrics.get("metrics/mAP50", None)),
                    "mAP50-95": metrics.get("metrics/mAP50-95(B)", metrics.get("metrics/mAP50-95", None)),
                    "precision": metrics.get("metrics/precision(B)", metrics.get("metrics/precision", None)),
                    "recall": metrics.get("metrics/recall(B)", metrics.get("metrics/recall", None)),
                    "params": metrics.get("model/parameters", None),
                    "flops": metrics.get("model/FLOPs", None),
                    "train_time": metrics.get("train/epochs", None),
                    "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                }
            )
            break

    if not found:
        print(f"⚠️ No results file found for {model_name}")

# Save consolidated results
if results:
    df = pd.DataFrame(results)
    df.to_csv("runs/ablation/ablation_results.csv", index=False)
    print("\n✅ Ablation study complete! Results saved to runs/ablation/ablation_results.csv")
    display(df)
else:
    print("⚠️ No results collected. Check that training has finished and results files exist.")

✅ Found results for 00_baseline: runs/ablation/00_baseline/results.csv
✅ Found results for 01_baseline_plus_efficientnet: runs/ablation/01_baseline_plus_efficientnet/results.csv
✅ Found results for 02_baseline_plus_fablocks: runs/ablation/02_baseline_plus_fablocks/results.csv
✅ Found results for 03_baseline_plus_hgblocks: runs/ablation/03_baseline_plus_hgblocks/results.csv
✅ Found results for 04_baseline_plus_spdadown: runs/ablation/04_baseline_plus_spdadown/results.csv
✅ Found results for 05_baseline_plus_dropblock: runs/ablation/05_baseline_plus_dropblock/results.csv
✅ Found results for 06_baseline_plus_cbam: runs/ablation/06_baseline_plus_cbam/results.csv
✅ Found results for 07_baseline_plus_bifpn: runs/ablation/07_baseline_plus_bifpn/results.csv
✅ Found results for 08_baseline_plus_context: runs/ablation/08_baseline_plus_context/results.csv
✅ Found results for 09_full_model: runs/ablation/09_full_model/results.csv

✅ Ablation study complete! Results saved to runs/ablation/ablation_

,model,mAP50,mAP50-95,precision,recall,params,flops,train_time,timestamp
0,00_baseline,0.44464,0.20561,0.62854,0.42215,None,None,None,2025-10-28 23:11:35
1,01_baseline_plus_efficientnet,0.95325,0.58955,0.95233,0.94569,None,None,None,2025-10-28 23:11:35
2,02_baseline_plus_fablocks,0.54605,0.25506,0.53705,0.61537,None,None,None,2025-10-28 23:11:35
3,03_baseline_plus_hgblocks,0.53280,0.22846,0.78011,0.43785,None,None,None,2025-10-28 23:11:35
4,04_baseline_plus_spdadown,0.40271,0.16252,0.36628,0.43530,None,None,None,2025-10-28 23:11:35
5,05_baseline_plus_dropblock,0.47637,0.19670,0.63896,0.46988,None,None,None,2025-10-28 23:11:35
6,06_baseline_plus_cbam,0.45944,0.18925,0.63473,0.43829,None,None,None,2025-10-28 23:11:35
7,07_baseline_plus_bifpn,0.47374,0.22154,0.41333,0.54235,None,None,None,2025-10-28 23:11:35
8,08_baseline_plus_context,0.48338,0.21567,0.64768,0.44861,None,None,None,2025-10-28 23:11:35
9,09_full_model,0.95214,0.61322,0.95415,0.93550,None,None,None,2025-10-28 23:11:35


In [15]:
# additive

import torch

from ultralytics.nn.tasks import DetectionModel

# cfg = "ultralytics/cfg/models/additive/00_baseline.yaml"
# cfg = "ultralytics/cfg/models/additive/01_baseline_plus_efficientnet.yaml"
# cfg = "ultralytics/cfg/models/additive/02_baseline_plus_fablocks.yaml"
# cfg = "ultralytics/cfg/models/additive/03_baseline_plus_hgblocks.yaml"
# cfg = "ultralytics/cfg/models/additive/04_baseline_plus_spdadown.yaml"
# cfg = "ultralytics/cfg/models/additive/05_baseline_plus_dropblock.yaml"
# cfg = "ultralytics/cfg/models/additive/06_baseline_plus_cbam.yaml"
# cfg = "ultralytics/cfg/models/additive/07_baseline_plus_bifpn.yaml"
# cfg = "ultralytics/cfg/models/additive/08_baseline_plus_context.yaml"
cfg = "ultralytics/cfg/models/additive/09_additive_full_model_Fusion_FIXED.yaml"
model = DetectionModel(cfg=cfg, nc=30)

x = torch.randn(1, 3, 640, 640)
y = model(x)
print("✅ Model runs successfully! Output shapes:")
for out in y if isinstance(y, list) else [y]:
    print(out.shape)

Overriding model.yaml nc=80 with nc=30

                   from  n    params  module                                       arguments                     
  0                  -1  1  10696232  ultralytics.nn.modules.efficientnet_backbone.EfficientNetBackbone['efficientnet_b3', True, [2, 3, 4]]
  1                  -1  1         0  ultralytics.nn.modules.hybrid_blocks.Adapter [1280, 256]                   
  2                  -1  1         0  ultralytics.nn.modules.hybrid_blocks.FABlock [256]                         
  3                  -1  1         0  ultralytics.nn.modules.hybrid_blocks.MyHGBlock[256, 256, 3, 1]              
  4                  -1  1         0  ultralytics.nn.modules.hybrid_blocks.SPDADown[384]                         
  5                  -1  1         0  ultralytics.nn.modules.hybrid_blocks.FABlock [384]                         
  6                  -1  1         0  ultralytics.nn.modules.hybrid_blocks.MyHGBlock[384, 384, 3, 1]              
  7                 

In [11]:
import os
from datetime import datetime

import pandas as pd
import yaml

from ultralytics import YOLO

# List of all additive config variants
configs = [
    "ultralytics/cfg/models/additive/00_baseline.yaml",
    "ultralytics/cfg/models/additive/01_baseline_plus_efficientnet.yaml",
    "ultralytics/cfg/models/additive/02_baseline_plus_fablocks.yaml",
    "ultralytics/cfg/models/additive/03_baseline_plus_hgblocks.yaml",
    "ultralytics/cfg/models/additive/04_baseline_plus_spdadown.yaml",
    "ultralytics/cfg/models/additive/05_baseline_plus_dropblock.yaml",
    "ultralytics/cfg/models/additive/06_baseline_plus_cbam.yaml",
    "ultralytics/cfg/models/additive/07_baseline_plus_bifpn.yaml",
    "ultralytics/cfg/models/additive/08_baseline_plus_context.yaml",
    "ultralytics/cfg/models/additive/09_additive_full_model_Fusion_FIXED.yaml",
]

results = []
DATASET = "charging-ports-7/data.yaml"
EPOCHS = 25
BATCH = 16
IMGSZ = 640
DEVICE = "0,1"  # booked GPUs

# Run all additive models
for cfg in configs:
    model_name = os.path.splitext(os.path.basename(cfg))[0]
    print(f"\n🚀 Training additive variant: {model_name}\n{'=' * 60}")

    model = YOLO(cfg)
    r = model.train(
        data=DATASET,
        epochs=EPOCHS,
        batch=BATCH,
        imgsz=IMGSZ,
        device=DEVICE,
        project="runs/additive",
        name=model_name,
        exist_ok=True,
    )

    # Locate results file (YAML, CSV, or JSON)
    base_path = f"runs/additive/{model_name}"
    possible_files = [
        os.path.join(base_path, "results.yaml"),
        os.path.join(base_path, "results.csv"),
        os.path.join(base_path, "results.json"),
    ]
    metrics = {}
    found = False

    for file in possible_files:
        if os.path.exists(file):
            found = True
            if file.endswith(".yaml"):
                with open(file) as f:
                    metrics = yaml.safe_load(f)
            elif file.endswith(".csv"):
                df_metrics = pd.read_csv(file)
                metrics = df_metrics.iloc[-1].to_dict()
            elif file.endswith(".json"):
                import json

                with open(file) as f:
                    metrics = json.load(f)
            break

    if found:
        results.append(
            {
                "model": model_name,
                "mAP50": metrics.get("metrics/mAP50(B)", metrics.get("metrics/mAP50", None)),
                "mAP50-95": metrics.get("metrics/mAP50-95(B)", metrics.get("metrics/mAP50-95", None)),
                "precision": metrics.get("metrics/precision(B)", metrics.get("metrics/precision", None)),
                "recall": metrics.get("metrics/recall(B)", metrics.get("metrics/recall", None)),
                "params": metrics.get("model/parameters", None),
                "flops": metrics.get("model/FLOPs", None),
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            }
        )
        print(f"✅ Metrics collected for {model_name}")
    else:
        print(f"⚠️ No results found for {model_name}")

# Save summary
df = pd.DataFrame(results)
df.to_csv("runs/additive/additive_results.csv", index=False)
print("\n✅ Additive study complete! Results saved to runs/additive/additive_results.csv")
display(df)


🚀 Training additive variant: 00_baseline
New https://pypi.org/project/ultralytics/8.3.222 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.195 🚀 Python-3.10.18 torch-2.7.1+cu118 CUDA:0 (NVIDIA RTX A6000, 48685MiB)
                                                        CUDA:1 (NVIDIA RTX A6000, 48685MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=charging-ports-7/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=

,model,mAP50,mAP50-95,precision,recall,params,flops,timestamp
0,00_baseline,0.44464,0.20561,0.62854,0.42215,None,None,2025-10-29 19:27:39
1,01_baseline_plus_efficientnet,0.95325,0.58955,0.95233,0.94569,None,None,2025-10-29 19:33:15
2,02_baseline_plus_fablocks,0.95808,0.60093,0.94776,0.89261,None,None,2025-10-29 19:39:26
3,03_baseline_plus_hgblocks,0.94887,0.59446,0.93402,0.95179,None,None,2025-10-29 19:45:48
4,04_baseline_plus_spdadown,0.96461,0.60482,0.95901,0.93949,None,None,2025-10-29 19:52:12
5,05_baseline_plus_dropblock,0.97353,0.59702,0.97025,0.93734,None,None,2025-10-29 19:58:35
6,06_baseline_plus_cbam,0.95459,0.60559,0.95516,0.95758,None,None,2025-10-29 20:05:08
7,07_baseline_plus_bifpn,0.96501,0.58724,0.96673,0.94559,None,None,2025-10-29 20:11:39
8,08_baseline_plus_context,0.94611,0.60193,0.94978,0.95159,None,None,2025-10-29 20:18:18
9,09_additive_full_model_Fusion_FIXED,0.94513,0.59913,0.94475,0.90914,None,None,2025-10-29 20:24:43


In [12]:
!pip install roboflow

from roboflow import Roboflow

rf = Roboflow(api_key="RUmK4wNPHx178VCDB6nH")
project = rf.workspace("caci-nr6wn").project("tools-detection-ce8so")
version = project.version(2)
dataset = version.download("yolov9")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to tools-detection-2 in yolov9:: 100%|████████████| 3228/3228 [00:00<00:00, 13837.08it/s]


In [13]:
import os
from datetime import datetime

import pandas as pd
import yaml

from ultralytics import YOLO

# List of all ablation config variants
configs = [
    "ultralytics/cfg/models/ablation/00_baseline.yaml",
    "ultralytics/cfg/models/ablation/01_baseline_plus_efficientnet.yaml",
    "ultralytics/cfg/models/ablation/02_baseline_plus_fablocks.yaml",
    "ultralytics/cfg/models/ablation/03_baseline_plus_hgblocks.yaml",
    "ultralytics/cfg/models/ablation/04_baseline_plus_spdadown.yaml",
    "ultralytics/cfg/models/ablation/05_baseline_plus_dropblock.yaml",
    "ultralytics/cfg/models/ablation/06_baseline_plus_cbam.yaml",
    "ultralytics/cfg/models/ablation/07_baseline_plus_bifpn.yaml",
    "ultralytics/cfg/models/ablation/08_baseline_plus_context.yaml",
    "ultralytics/cfg/models/ablation/09_full_model.yaml",
]

results = []
DATASET = "tools-detection-2/data.yaml"
EPOCHS = 25
BATCH = 16
IMGSZ = 640
DEVICE = "0,1"  # booked GPUs

# Run all additive models
for cfg in configs:
    model_name = os.path.splitext(os.path.basename(cfg))[0]
    print(f"\n🚀 Training additive variant: {model_name}\n{'=' * 60}")

    model = YOLO(cfg)
    r = model.train(
        data=DATASET,
        epochs=EPOCHS,
        batch=BATCH,
        imgsz=IMGSZ,
        device=DEVICE,
        project="runs/additive_tool_detection",
        name=model_name,
        exist_ok=True,
    )

    # Locate results file (YAML, CSV, or JSON)
    base_path = f"runs/additive_tool_detection/{model_name}"
    possible_files = [
        os.path.join(base_path, "results.yaml"),
        os.path.join(base_path, "results.csv"),
        os.path.join(base_path, "results.json"),
    ]
    metrics = {}
    found = False

    for file in possible_files:
        if os.path.exists(file):
            found = True
            if file.endswith(".yaml"):
                with open(file) as f:
                    metrics = yaml.safe_load(f)
            elif file.endswith(".csv"):
                df_metrics = pd.read_csv(file)
                metrics = df_metrics.iloc[-1].to_dict()
            elif file.endswith(".json"):
                import json

                with open(file) as f:
                    metrics = json.load(f)
            break

    if found:
        results.append(
            {
                "model": model_name,
                "mAP50": metrics.get("metrics/mAP50(B)", metrics.get("metrics/mAP50", None)),
                "mAP50-95": metrics.get("metrics/mAP50-95(B)", metrics.get("metrics/mAP50-95", None)),
                "precision": metrics.get("metrics/precision(B)", metrics.get("metrics/precision", None)),
                "recall": metrics.get("metrics/recall(B)", metrics.get("metrics/recall", None)),
                "params": metrics.get("model/parameters", None),
                "flops": metrics.get("model/FLOPs", None),
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            }
        )
        print(f"✅ Metrics collected for {model_name}")
    else:
        print(f"⚠️ No results found for {model_name}")

# Save summary
df = pd.DataFrame(results)
df.to_csv("runs/additive_tool_detection/additive_results.csv", index=False)
print("\n✅ Additive study complete! Results saved to runs/additive/additive_results.csv")
display(df)


🚀 Training additive variant: 00_baseline
New https://pypi.org/project/ultralytics/8.3.222 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.195 🚀 Python-3.10.18 torch-2.7.1+cu118 CUDA:0 (NVIDIA RTX A6000, 48685MiB)
                                                        CUDA:1 (NVIDIA RTX A6000, 48685MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=tools-detection-2/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup

,model,mAP50,mAP50-95,precision,recall,params,flops,timestamp
0,00_baseline,0.00982,0.00188,0.03781,0.04288,None,None,2025-10-29 21:07:30
1,01_baseline_plus_efficientnet,0.83221,0.46546,0.77807,0.79794,None,None,2025-10-29 21:14:57
2,02_baseline_plus_fablocks,0.02066,0.00352,0.40488,0.05223,None,None,2025-10-29 21:47:43
3,03_baseline_plus_hgblocks,0.01595,0.00290,0.04411,0.04592,None,None,2025-10-29 22:10:15
4,04_baseline_plus_spdadown,0.00781,0.00153,0.03582,0.03034,None,None,2025-10-29 22:24:53
5,05_baseline_plus_dropblock,0.01121,0.00205,0.03983,0.03744,None,None,2025-10-29 22:39:57
6,06_baseline_plus_cbam,0.01441,0.00336,0.05061,0.04187,None,None,2025-10-29 22:56:31
7,07_baseline_plus_bifpn,0.01048,0.00198,0.04740,0.04762,None,None,2025-10-29 23:11:45
8,08_baseline_plus_context,0.00835,0.00173,0.03773,0.05012,None,None,2025-10-29 23:27:29
9,09_full_model,0.85396,0.48544,0.80896,0.80148,None,None,2025-10-29 23:36:20


In [1]:
import os
from datetime import datetime

import pandas as pd
import yaml

from ultralytics import YOLO

# List of all additive config variants
configs = [
    "ultralytics/cfg/models/additive/00_baseline.yaml",
    "ultralytics/cfg/models/additive/01_baseline_plus_efficientnet.yaml",
    "ultralytics/cfg/models/additive/02_baseline_plus_fablocks.yaml",
    "ultralytics/cfg/models/additive/03_baseline_plus_hgblocks.yaml",
    "ultralytics/cfg/models/additive/04_baseline_plus_spdadown.yaml",
    "ultralytics/cfg/models/additive/05_baseline_plus_dropblock.yaml",
    "ultralytics/cfg/models/additive/06_baseline_plus_cbam.yaml",
    "ultralytics/cfg/models/additive/07_baseline_plus_bifpn.yaml",
    "ultralytics/cfg/models/additive/08_baseline_plus_context.yaml",
    "ultralytics/cfg/models/additive/09_additive_full_model_Fusion_FIXED.yaml",
]

results = []
DATASET = "tools-detection-2/data.yaml"
EPOCHS = 25
BATCH = 16
IMGSZ = 640
DEVICE = "0,1"  # booked GPUs

# Run all additive models
for cfg in configs:
    model_name = os.path.splitext(os.path.basename(cfg))[0]
    print(f"\n🚀 Training additive variant: {model_name}\n{'=' * 60}")

    model = YOLO(cfg)
    r = model.train(
        data=DATASET,
        epochs=EPOCHS,
        batch=BATCH,
        imgsz=IMGSZ,
        device=DEVICE,
        project="runs/ablation_tool_detection",
        name=model_name,
        exist_ok=True,
    )

    # Locate results file (YAML, CSV, or JSON)
    base_path = f"runs/ablation_tool_detection/{model_name}"
    possible_files = [
        os.path.join(base_path, "results.yaml"),
        os.path.join(base_path, "results.csv"),
        os.path.join(base_path, "results.json"),
    ]
    metrics = {}
    found = False

    for file in possible_files:
        if os.path.exists(file):
            found = True
            if file.endswith(".yaml"):
                with open(file) as f:
                    metrics = yaml.safe_load(f)
            elif file.endswith(".csv"):
                df_metrics = pd.read_csv(file)
                metrics = df_metrics.iloc[-1].to_dict()
            elif file.endswith(".json"):
                import json

                with open(file) as f:
                    metrics = json.load(f)
            break

    if found:
        results.append(
            {
                "model": model_name,
                "mAP50": metrics.get("metrics/mAP50(B)", metrics.get("metrics/mAP50", None)),
                "mAP50-95": metrics.get("metrics/mAP50-95(B)", metrics.get("metrics/mAP50-95", None)),
                "precision": metrics.get("metrics/precision(B)", metrics.get("metrics/precision", None)),
                "recall": metrics.get("metrics/recall(B)", metrics.get("metrics/recall", None)),
                "params": metrics.get("model/parameters", None),
                "flops": metrics.get("model/FLOPs", None),
                "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            }
        )
        print(f"✅ Metrics collected for {model_name}")
    else:
        print(f"⚠️ No results found for {model_name}")

# Save summary
df = pd.DataFrame(results)
df.to_csv("runs/ablation_tool_detection/additive_results.csv", index=False)
print("\n✅ Additive study complete! Results saved to runs/additive/additive_results.csv")
display(df)


🚀 Training additive variant: 00_baseline
New https://pypi.org/project/ultralytics/8.3.223 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.195 🚀 Python-3.10.18 torch-2.7.1+cu118 CUDA:0 (NVIDIA RTX A6000, 48685MiB)
                                                        CUDA:1 (NVIDIA RTX A6000, 48685MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=tools-detection-2/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup

,model,mAP50,mAP50-95,precision,recall,params,flops,timestamp
0,00_baseline,0.00982,0.00188,0.03781,0.04288,None,None,2025-10-30 20:29:58
1,01_baseline_plus_efficientnet,0.83221,0.46546,0.77807,0.79794,None,None,2025-10-30 20:37:42
2,02_baseline_plus_fablocks,0.85878,0.47101,0.82732,0.80814,None,None,2025-10-30 20:46:11
3,03_baseline_plus_hgblocks,0.83967,0.48889,0.78563,0.80564,None,None,2025-10-30 20:54:55
4,04_baseline_plus_spdadown,0.85396,0.48451,0.81626,0.80630,None,None,2025-10-30 21:03:50
5,05_baseline_plus_dropblock,0.84265,0.46313,0.81291,0.78709,None,None,2025-10-30 21:12:45
6,06_baseline_plus_cbam,0.83228,0.47641,0.81611,0.78379,None,None,2025-10-30 21:21:44
7,07_baseline_plus_bifpn,0.82687,0.46587,0.80927,0.77426,None,None,2025-10-30 21:30:44
8,08_baseline_plus_context,0.83727,0.46849,0.80150,0.80071,None,None,2025-10-30 21:39:57
9,09_additive_full_model_Fusion_FIXED,0.83544,0.45868,0.80843,0.82314,None,None,2025-10-30 21:48:44


In [3]:
import os
from datetime import datetime

import pandas as pd
import yaml

from ultralytics import YOLO

DATASET = "tools-detection-2/data.yaml"
EPOCHS = 125
BATCH = 16
IMGSZ = 640
DEVICE = "0,1"  # booked GPUs

model = YOLO("ultralytics/cfg/models/additive/09_additive_full_model_Fusion_FIXED.yaml")
r = model.train(
    data=DATASET,
    epochs=EPOCHS,
    batch=BATCH,
    imgsz=IMGSZ,
    device=DEVICE,
    project="runs/additive_final",
    name=model_name,
    exist_ok=True,
)

New https://pypi.org/project/ultralytics/8.3.223 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.195 🚀 Python-3.10.18 torch-2.7.1+cu118 CUDA:0 (NVIDIA RTX A6000, 48685MiB)
                                                        CUDA:1 (NVIDIA RTX A6000, 48685MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=tools-detection-2/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=125, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=ultralytics/cfg/m

In [3]:
import os
from datetime import datetime

import pandas as pd
import yaml

from ultralytics import YOLO

DATASET = "tools-detection-2/data.yaml"
EPOCHS = 125
BATCH = 16
IMGSZ = 640
DEVICE = "0,1"

model = YOLO("ultralytics/cfg/models/ablation/09_full_model.yaml")
r = model.train(
    data=DATASET,
    epochs=EPOCHS,
    batch=BATCH,
    imgsz=IMGSZ,
    device=DEVICE,
    project="runs/ablation_final",
    name="ablation_full_model",
    exist_ok=True,
)

New https://pypi.org/project/ultralytics/8.3.223 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.195 🚀 Python-3.10.18 torch-2.7.1+cu118 CUDA:0 (NVIDIA RTX A6000, 48685MiB)
                                                        CUDA:1 (NVIDIA RTX A6000, 48685MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=tools-detection-2/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=125, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=ultralytics/cfg/m

W1031 01:00:41.100000 605451 /home/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/site-packages/torch/distributed/elastic/multiprocessing/api.py:900] Sending process 605458 closing signal SIGTERM
E1031 01:00:41.721000 605451 /home/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/site-packages/torch/distributed/elastic/multiprocessing/api.py:874] failed (exitcode: -9) local_rank: 0 (pid: 605457) of binary: /users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/bin/python
Traceback (most recent call last):
  File "/users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/site-packages/torch/distributed/run.py", line 89

CalledProcessError: Command '['/users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/bin/python', '-m', 'torch.distributed.run', '--nproc_per_node', '2', '--master_port', '37673', '/users/2/250027044/.config/Ultralytics/DDP/_temp_hv9q81o0140268097076960.py']' returned non-zero exit status 1.

In [4]:
import torch

from ultralytics.nn.tasks import DetectionModel

cfg = "ultralytics/cfg/models/custom/custom_yolov9_outperform.yaml"
model = DetectionModel(cfg=cfg, nc=30)
x = torch.randn(1, 3, 640, 640)
y = model(x)
print("✅ Model loaded and forward passed successfully!")

Overriding model.yaml nc=80 with nc=30

                   from  n    params  module                                       arguments                     
  0                  -1  1  10696232  ultralytics.nn.modules.efficientnet_backbone.EfficientNetBackbone['efficientnet_b3', True, [2, 3, 4]]
  1                  -1  1         0  ultralytics.nn.modules.hybrid_blocks.Adapter [1280, 256]                   
  2                  -1  1         0  ultralytics.nn.modules.hybrid_blocks.FABlock [256]                         
  3                  -1  1         0  ultralytics.nn.modules.hybrid_blocks.MyHGBlock[256, 256, 3, 1]              
  4                  -1  1    885504  ultralytics.nn.modules.conv.Conv             [384, 3, 2]                   
  5                  -1  1         0  ultralytics.nn.modules.hybrid_blocks.FABlock [384]                         
  6                  -1  1         0  ultralytics.nn.modules.hybrid_blocks.MyHGBlock[384, 384, 3, 1]              
  7                 

In [5]:
import os
from datetime import datetime

import pandas as pd
import yaml

from ultralytics import YOLO

DATASET = "tools-detection-2/data.yaml"
EPOCHS = 125
BATCH = 16
IMGSZ = 640
DEVICE = "0,1"  # booked GPUs

model = YOLO("ultralytics/cfg/models/custom/custom_yolov9_outperform.yaml")
r = model.train(
    data=DATASET,
    epochs=EPOCHS,
    batch=BATCH,
    imgsz=IMGSZ,
    device=DEVICE,
    project="runs/ablation_final_outperform",
    name="ablation_full_model_outperform",
    exist_ok=True,
)

New https://pypi.org/project/ultralytics/8.3.223 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.195 🚀 Python-3.10.18 torch-2.7.1+cu118 CUDA:0 (NVIDIA RTX A6000, 48685MiB)
                                                        CUDA:1 (NVIDIA RTX A6000, 48685MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=tools-detection-2/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=125, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=ultralytics/cfg/m

In [ ]:
import os
from datetime import datetime

import pandas as pd
import yaml

from ultralytics import YOLO

DATASET = "tools-detection-2/data.yaml"
EPOCHS = 125
BATCH = 16
IMGSZ = 640
DEVICE = "0,1"  # booked GPUs

model = YOLO("ultralytics/cfg/models/additive/03_baseline_plus_hgblocks.yaml")
r = model.train(
    data=DATASET,
    epochs=EPOCHS,
    batch=BATCH,
    imgsz=IMGSZ,
    device=DEVICE,
    project="runs/ablation_final_outperform",
    name="ablation_full_model_outperform",
    exist_ok=True,
)

New https://pypi.org/project/ultralytics/8.3.227 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.195 🚀 Python-3.10.18 torch-2.7.1+cu118 CUDA:0 (NVIDIA RTX A6000, 48685MiB)
                                                        CUDA:1 (NVIDIA RTX A6000, 48685MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=tools-detection-2/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=125, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=ultralytics/cfg/m

In [6]:
import os
from datetime import datetime

from ultralytics import YOLO

# --- Constants ---
DATASET = "tools-detection-2/data.yaml"
MODEL_YAML = "ultralytics/cfg/models/additive/09_additive_full_model_Fusion_FIXED.yaml"
EPOCHS = 125
BATCH = 16
IMGSZ = 640
DEVICE = "0,1"
PROJECT_NAME = "runs/additive_final_tuning"

# --- Hyperparameter Grid ---

hp_grid = [
    {"name": "AdamW_lr_0.005_wd_0.0007", "optimizer": "AdamW", "lr0": 0.005, "weight_decay": 0.0007},
    {"name": "AdamW_lr_0.002_wd_0.0007", "optimizer": "AdamW", "lr0": 0.002, "weight_decay": 0.0007},
    {
        "name": "SGD_lr_0.005_momentum_0.9",
        "optimizer": "SGD",
        "lr0": 0.005,
        "momentum": 0.9,
        "weight_decay": 0.0005,  # SGD uses default weight decay
    },
]

# --- 3. The Tuning Loop ---
print(f"Starting hyperparameter sweep... Found {len(hp_grid)} runs.")

for hps in hp_grid:
    model = YOLO(MODEL_YAML)

    run_name = hps.pop("name", "default_run_name")

    run_epochs = hps.pop("epochs", EPOCHS)

    print("\n" + "=" * 30)
    print(f"STARTING RUN: {run_name}")
    print(f"EPOCHS: {run_epochs}")
    print(f"HYPERPARAMETERS: {hps}")
    print("=" * 30 + "\n")

    try:
        r = model.train(
            data=DATASET,
            epochs=run_epochs,
            batch=BATCH,
            imgsz=IMGSZ,
            device=DEVICE,
            project=PROJECT_NAME,
            name=run_name,
            exist_ok=True,
            **hps,
        )
        print(f"\n--- FINISHED RUN: {run_name} ---")

    except Exception as e:
        print(f"\n--- RUN FAILED: {run_name} ---")
        print(f"Error: {e}")
        print("Continuing to the next run...")

print("\n" + "=" * 30)
print("Hyperparameter sweep complete.")
print(f"All results saved to: {PROJECT_NAME}")
print("=" * 30)

Starting hyperparameter sweep... Found 3 runs.

STARTING RUN: AdamW_lr_0.005_wd_0.0007
EPOCHS: 125
HYPERPARAMETERS: {'optimizer': 'AdamW', 'lr0': 0.005, 'weight_decay': 0.0007}

New https://pypi.org/project/ultralytics/8.3.223 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.195 🚀 Python-3.10.18 torch-2.7.1+cu118 CUDA:0 (NVIDIA RTX A6000, 48685MiB)
                                                        CUDA:1 (NVIDIA RTX A6000, 48685MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=tools-detection-2/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=125, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7

In [2]:
import os

from ultralytics import YOLO

# --- 1. Constants (Set these once) ---
DATASET = "tools-detection-2/data.yaml"
MODEL_YAML = "ultralytics/cfg/models/additive/09_additive_full_model_Fusion_FIXED.yaml"
MODEL_B4_YAML = "ultralytics/cfg/models/additive/09_full_model_B4_FIXED.yaml"  # You will need to create this
BATCH = 16
IMGSZ = 640
DEVICE = "0,1"
PROJECT_NAME = "runs/additive_final_push"  # New project folder

# --- 2. Create the B4 Model YAML ---
# (This code creates the B4-YAML file for you automatically)
try:
    with open(MODEL_YAML) as f:
        model_b3_config = f.read()

    model_b4_config = model_b3_config.replace("efficientnet_b3", "efficientnet_b4")

    with open(MODEL_B4_YAML, "w") as f:
        f.write(model_b4_config)
    print(f"Successfully created {MODEL_B4_YAML}")
except Exception as e:
    print(f"Error creating B4 YAML: {e}. Please create it manually.")

# --- 3. Define Your Final Experiments ---
# This is no longer a grid search, but a sequence of your best ideas.
final_runs = [
    {
        "name": "01_Full_300_Epochs",
        "model_yaml": MODEL_YAML,
        "epochs": 300,
        "lr0": 0.01,  # Your proven best LR
    },
    {
        "name": "02_Full_300_Epochs_With_Augs",
        "model_yaml": MODEL_YAML,
        "epochs": 300,
        "lr0": 0.01,
        "mixup": 0.1,
        "copy_paste": 0.1,
    },
    {
        "name": "03_Full_B4_Backbone_300_Epochs",
        "model_yaml": MODEL_B4_YAML,  # Use the new B4 model
        "epochs": 300,
        "lr0": 0.01,  # Start with 0.01, but B4 may also need lr0=0.005
    },
]

# --- 4. The Training Loop ---
print(f"Starting final push... Found {len(final_runs)} runs.")

for hps in final_runs:
    run_name = hps.pop("name")
    model_file = hps.pop("model_yaml")
    run_epochs = hps.pop("epochs", 125)  # Default to 125 if not set

    # Re-initialize the model from its specific YAML
    model = YOLO(model_file)

    print("\n" + "=" * 30)
    print(f"STARTING RUN: {run_name}")
    print(f"MODEL: {model_file}")
    print(f"EPOCHS: {run_epochs}")
    print(f"HYPERPARAMETERS: {hps}")
    print("=" * 30 + "\n")

    try:
        r = model.train(
            data=DATASET,
            epochs=run_epochs,
            batch=BATCH,
            imgsz=IMGSZ,
            device=DEVICE,
            project=PROJECT_NAME,
            name=run_name,
            exist_ok=True,
            **hps,  # Pass all other HPs
        )
        print(f"\n--- FINISHED RUN: {run_name} ---")

    except Exception as e:
        print(f"\n--- RUN FAILED: {run_name} ---")
        print(f"Error: {e}")
        print("Continuing to the next run...")

print("\n" + "=" * 30)
print("Final experiments complete.")
print(f"All results saved to: {PROJECT_NAME}")
print("=" * 30)

Successfully created ultralytics/cfg/models/additive/09_full_model_B4_FIXED.yaml
Starting final push... Found 3 runs.

STARTING RUN: 01_Full_300_Epochs
MODEL: ultralytics/cfg/models/additive/09_additive_full_model_Fusion_FIXED.yaml
EPOCHS: 300
HYPERPARAMETERS: {'lr0': 0.01}

New https://pypi.org/project/ultralytics/8.3.223 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.195 🚀 Python-3.10.18 torch-2.7.1+cu118 CUDA:0 (NVIDIA RTX A6000, 48685MiB)
                                                        CUDA:1 (NVIDIA RTX A6000, 48685MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=tools-detection-2/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=True, fliplr=

W1031 23:46:37.750000 1066091 /home/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/site-packages/torch/distributed/elastic/multiprocessing/api.py:900] Sending process 1066102 closing signal SIGTERM
E1031 23:46:38.268000 1066091 /home/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/site-packages/torch/distributed/elastic/multiprocessing/api.py:874] failed (exitcode: -9) local_rank: 0 (pid: 1066101) of binary: /users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/bin/python
Traceback (most recent call last):
  File "/users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/site-packages/torch/distributed/run.py", lin


--- RUN FAILED: 01_Full_300_Epochs ---
Error: Command '['/users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/bin/python', '-m', 'torch.distributed.run', '--nproc_per_node', '2', '--master_port', '36205', '/users/2/250027044/.config/Ultralytics/DDP/_temp_cyv2dar8139880472355472.py']' returned non-zero exit status 1.
Continuing to the next run...

STARTING RUN: 02_Full_300_Epochs_With_Augs
MODEL: ultralytics/cfg/models/additive/09_additive_full_model_Fusion_FIXED.yaml
EPOCHS: 300
HYPERPARAMETERS: {'lr0': 0.01, 'mixup': 0.1, 'copy_paste': 0.1}

New https://pypi.org/project/ultralytics/8.3.223 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.195 🚀 Python-3.10.18 torch-2.7.1+cu118 CUDA:0 (NVIDIA RTX A6000, 48685MiB)
                                                        CUDA:1 (NVIDIA RTX A6000, 48685MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_

W1031 23:47:47.736000 1066448 /home/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/site-packages/torch/distributed/elastic/multiprocessing/api.py:900] Sending process 1066453 closing signal SIGTERM
E1031 23:47:48.254000 1066448 /home/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/site-packages/torch/distributed/elastic/multiprocessing/api.py:874] failed (exitcode: -9) local_rank: 0 (pid: 1066452) of binary: /users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/bin/python
Traceback (most recent call last):
  File "/users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/site-packages/torch/distributed/run.py", lin


--- RUN FAILED: 02_Full_300_Epochs_With_Augs ---
Error: Command '['/users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/bin/python', '-m', 'torch.distributed.run', '--nproc_per_node', '2', '--master_port', '59555', '/users/2/250027044/.config/Ultralytics/DDP/_temp_c4o2lbvf139880470417168.py']' returned non-zero exit status 1.
Continuing to the next run...

STARTING RUN: 03_Full_B4_Backbone_300_Epochs
MODEL: ultralytics/cfg/models/additive/09_full_model_B4_FIXED.yaml
EPOCHS: 300
HYPERPARAMETERS: {'lr0': 0.01}

New https://pypi.org/project/ultralytics/8.3.223 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.195 🚀 Python-3.10.18 torch-2.7.1+cu118 CUDA:0 (NVIDIA RTX A6000, 48685MiB)
                                                        CUDA:1 (NVIDIA RTX A6000, 48685MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, cop

W1031 23:48:57.876000 1066986 /home/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/site-packages/torch/distributed/elastic/multiprocessing/api.py:900] Sending process 1066994 closing signal SIGTERM
E1031 23:48:58.443000 1066986 /home/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/site-packages/torch/distributed/elastic/multiprocessing/api.py:874] failed (exitcode: -9) local_rank: 0 (pid: 1066993) of binary: /users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/bin/python
Traceback (most recent call last):
  File "/users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/lib/python3.10/site-packages/torch/distributed/run.py", lin


--- RUN FAILED: 03_Full_B4_Backbone_300_Epochs ---
Error: Command '['/users/2/250027044/anaconda3/envs/yolov9_custom_ultralytics/bin/python', '-m', 'torch.distributed.run', '--nproc_per_node', '2', '--master_port', '46185', '/users/2/250027044/.config/Ultralytics/DDP/_temp_xpoyv_1_139880296076128.py']' returned non-zero exit status 1.
Continuing to the next run...

Final experiments complete.
All results saved to: runs/additive_final_push


In [11]:
!pip install roboflow

from roboflow import Roboflow

rf = Roboflow(api_key="RUmK4wNPHx178VCDB6nH")
project = rf.workspace("academic-lsrwt").project("ev-battery-components-edfw3")
version = project.version(2)
dataset = version.download("yolov9")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Ev-Battery-Components-2 in yolov9:: 100%|██████| 4600/4600 [00:00<00:00, 11850.75it/s]
